# Previsão se um aluno vai tirar >= 7 na prova com base no teste

In [1]:
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

## Classe de regressão logística
*Faz o trabalho pesado em regressão logística.*

In [2]:
class LogisticRegression:
    def __init__(self, learning_rate=0.001, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights = np.zeros(n_features)
        self.bias = 0

        # gradient descent
        for _ in range(self.n_iters):
            # approximate y with linear combination of weights and x, plus bias
            linear_model = np.dot(X, self.weights) + self.bias
            # apply sigmoid function
            y_predicted = self._sigmoid(linear_model)

            # compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)
            # update parameters
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        
        return np.array(y_predicted_cls)
    
    def plot(self, X, y, legend):
        # only plots if X refers to exactly 2 variables
        if X.shape[1] != 2:
            raise ValueError("Can plot only for X's that refers to exactly 2 vars.")
        
        slope = -(self.weights[0]/self.weights[1])
        intercept = -(self.bias/self.weights[1])
        predictions = self.predict(X)

        sns.set_style('white')
        sns.scatterplot(x = X[:,0], y= X[:,1], hue=y.reshape(-1), style=predictions.reshape(-1));

        ax = plt.gca()
        ax.autoscale(False)
        x_vals = np.array(ax.get_xlim())
        y_vals = intercept + (slope * x_vals)
        plt.plot(x_vals, y_vals, c="k");
        
        plt.xlabel(legend[0])
        plt.ylabel(legend[1])

    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

## Importando a base
*Importa a base, seleciona as variáveis, limpa, normaliza.*


In [113]:
df = pd.read_csv("db_estudantes.csv")

df.sample(5)

,school,school_setting,school_type,classroom,teaching_method,n_student,student_id,gender,lunch,pretest,posttest
1509,UKPGS,Suburban,Public,9AW,Standard,25.0,NMN96,Female,Does not qualify,87.0,92.0
1879,VVTVA,Urban,Public,A93,Experimental,30.0,HY8JN,Male,Qualifies for reduced/free lunch,29.0,39.0
1598,UUUQX,Suburban,Non-public,6C1,Standard,16.0,0KUVX,Female,Does not qualify,62.0,75.0
790,GOOBU,Urban,Public,U6J,Standard,25.0,3TJ3A,Female,Qualifies for reduced/free lunch,43.0,53.0
284,CUQAM,Urban,Public,OMI,Standard,28.0,2EQ4L,Male,Does not qualify,61.0,62.0


In [114]:
def booleamize(x):
	return round((x-20)/100)

df['posttest'] = df['posttest'].apply(booleamize)
# ======================================================== #
# =================== Seleção de Dados =================== #
# ======================================================== #
dados = {
    'X' : ['pretest'],
    'y' : 'posttest',
    'normalizada' : True
}
# ======================================================== #


df = df[ dados['X']+[dados['y']] ]
df = df.dropna()

if not dados['normalizada']:
    for col in dados['X']:
        df[[col]] = df[[col]]/df[[col]].mean()

X = df[ dados['X'] ].to_numpy()
y = df[[ dados['y'] ]].to_numpy()
y = np.hstack((y)).T

df.sample(5)

,pretest,posttest
1642,75.0,1
1777,63.0,1
1826,51.0,0
883,75.0,1
154,54.0,0


## Aplicando a regressão logística
*Aplica a função e avalia a precisão da previsão.*

In [115]:
regressor = LogisticRegression(learning_rate=0.001, n_iters=20000)
regressor.fit(X, y)
predictions = regressor.predict(X)

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

print(f"A precisão do modelo é: {accuracy(y, predictions)}")

A precisão do modelo é: 0.8931082981715893


# Plotando os resultados visualmente
*~ se você estiver analisando exatamente 2 vars numéricas ~*

In [110]:
try:
    regressor.plot(X, y, dados['X'])
except:
    print("Sem visualização disponível.")

Sem visualização disponível.


## Mostrando os pesos
*Cria uma tabela mostrando os pesos de avaliação das variáveis numéricas.*

In [111]:
norma_pesos = pd.DataFrame(regressor.weights)/pd.DataFrame(regressor.weights).abs().sum()
norma_pesos = norma_pesos[0].values.tolist()

dfpesos = pd.DataFrame({'Pesos(%)':norma_pesos}, index=dados['X'])

dfpesos

,Pesos(%)
pretest,1.0
